In [1]:
import os, numpy as np, ipywidgets as widgets

from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from ipywidgets import HBox, VBox
import ipywidgets as widgets

import render_filtered as rend

In [2]:
def plot_image(name, lms):
    plt.imshow(rend.linrgb_to_srgb(rend.load_adapted_image_linrgb(name, lms)))

In [3]:
sliders = [widgets.FloatSlider(min=0.0, max=1.0, value=1.0, continuous_update=False, description=f'{nm} nm')
     for nm in rend.IMG_WAVELENS]
enable_filter = widgets.Checkbox(value=True, description="Enable filter")
enable_shift = widgets.Checkbox(value=True, description="Enable shift")
image_chooser = widgets.Dropdown(options=sorted(os.listdir('data/')), description="Image")
lshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="L (red) shift")
mshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="M (green) shift")
sshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="S (blue) shift")
lshift.layout.width = '780px'
mshift.layout.width = '780px'
sshift.layout.width = '780px'
widget_dict = {'enable_filter': enable_filter, 'enable_shift': enable_shift,
               'name': image_chooser,
               'lshift': lshift, 'mshift': mshift, 'sshift': sshift,
               **{s.description: s for s in sliders}}

In [4]:
def plot_lms(lmsp):
    fig, ax = plt.subplots(figsize=(12, 3))
    (l,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 0] / max(lmsp[:, 0]), 'r')
    (m,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 1] / max(lmsp[:, 1]), 'g')
    (s,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 2] / max(lmsp[:, 2]), 'b')
    ax.set_xlim((rend.PREC_WAVELENS.min(), 700))
    ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.xaxis.set_minor_locator(MultipleLocator(10))
    ax.set_xlabel('Wavelength (nm)')
    return l, m, s

In [5]:
def calc_lms(*, lshift, mshift, sshift, enable_filter, enable_shift, **kwargs):
    lms = rend.LMS_STD.copy()
    if enable_shift:
        lms = rend.shift_lms(lms, lshift, mshift, sshift)
    if enable_filter:
        lms *= rend.interp_freqs(np.asarray([s.value for s in sliders])).reshape(-1, 1)
    return lms

def update_image(*, name, **kwargs):
    plt.figure(figsize=(8, 8))
    plot_image(name, calc_lms(**kwargs))

def update_lms(**kwargs):
    plot_lms(calc_lms(**kwargs))

image = widgets.interactive_output(update_image, widget_dict)

lms_plot = widgets.interactive_output(update_lms, widget_dict)
lms_plot.layout.height = '220px'
lms_plot.layout.width = '800px'

sliders_box = HBox([VBox(sliders[:10]), VBox(sliders[10:20]), VBox(sliders[20:])])

lms_box = VBox([lms_plot, lshift, mshift, sshift])
layout = VBox([HBox([image, VBox([image_chooser, enable_filter, enable_shift])]), sliders_box])

In [6]:
disp = display(lms_box)

In [8]:
display(layout)